In [132]:
from ouster.sdk import open_source
from ouster.sdk.mapping.slam import KissBackend
import ouster.sdk.client as client
import numpy as np

# Open the PCAP file
source_file_path = r"C:\Users\Bryan\Desktop\ece484\Final_Project\ouster_pcap\2024117_1023_OS-1-128_122423000263.pcap"
data_source = open_source(source_file_path, sensor_idx=-1)

# Initialize SLAM
slam = KissBackend(data_source.metadata, max_range=75, min_range=1, voxel_size=1.0)

# Create xyz lookup table
xyz_lut = client.XYZLut(data_source.metadata[0])

# Initialize parameters
n_channels = data_source.metadata[0].format.pixels_per_column
n_points = data_source.metadata[0].format.columns_per_frame
n_attributes = 7

# Count frames and reinitialize
frame_count = sum(1 for _ in data_source)
data_source = open_source(source_file_path, sensor_idx=-1)

# Initialize output array
lidar_data = np.zeros((frame_count, n_channels, n_points, n_attributes))

# Process each frame
for frame_idx, scans in enumerate(data_source):
    # Update SLAM
    scans_w_poses = slam.update(scans)
    if not scans_w_poses:
        continue
    
    scan = scans_w_poses[0]
    
    # Get xyz points in sensor frame
    xyz_points = xyz_lut(scan)  # (H x W x 3)
    
    # Get poses from SLAM for global coordinates
    poses = scan.pose  # List of 1024 poses (one per column)
    
    # Transform points using SLAM poses
    xyz_global = np.zeros_like(xyz_points)
    for col in range(xyz_points.shape[1]):
        pose = poses[col]  # Get pose for this column
        points = xyz_points[:, col, :]  # Get points for this column (H x 3)
        # Apply transformation: R * points + t
        xyz_global[:, col, :] = (pose[:3, :3] @ points.T).T + pose[:3, 3]
    
    # Get other fields
    ranges = scan.field(client.ChanField.RANGE)
    reflectivity = scan.field(client.ChanField.REFLECTIVITY)
    signal = scan.field(client.ChanField.SIGNAL)
    near_ir = scan.field(client.ChanField.NEAR_IR)
    
    # Store data
    lidar_data[frame_idx, :, :, 0:3] = xyz_global  # Global coordinates from SLAM
    lidar_data[frame_idx, :, :, 3] = reflectivity
    lidar_data[frame_idx, :, :, 4] = ranges
    lidar_data[frame_idx, :, :, 5] = signal
    lidar_data[frame_idx, :, :, 6] = near_ir
    
    if frame_idx % 10 == 0:
        print(f"Processed frame {frame_idx}/{frame_count}")

np.save("lidar_structured_data_global.npy", lidar_data)

INFO:root:Kiss-ICP voxel map size is 1 m


loading metadata from ['C:\\Users\\Bryan\\Desktop\\ece484\\Final_Project\\ouster_pcap\\2024117_1023_OS-1-128_122423000263.json']
loading metadata from ['C:\\Users\\Bryan\\Desktop\\ece484\\Final_Project\\ouster_pcap\\2024117_1023_OS-1-128_122423000263.json']
Processed frame 0/408
Processed frame 10/408
Processed frame 20/408
Processed frame 30/408
Processed frame 40/408
Processed frame 50/408
Processed frame 60/408
Processed frame 70/408
Processed frame 80/408
Processed frame 90/408
Processed frame 100/408
Processed frame 110/408
Processed frame 120/408
Processed frame 130/408
Processed frame 140/408
Processed frame 150/408
Processed frame 160/408
Processed frame 170/408
Processed frame 180/408
Processed frame 190/408
Processed frame 200/408
Processed frame 210/408
Processed frame 220/408
Processed frame 230/408
Processed frame 240/408
Processed frame 250/408
Processed frame 260/408
Processed frame 270/408
Processed frame 280/408
Processed frame 290/408
Processed frame 300/408
Processed

In [133]:
lidar_data.shape

(408, 128, 1024, 7)